# West Virginia Production dataset - WVGES version



In [1]:
# python/pandas preamble
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import zipfile, os

# different sets contain input files that are formatted similarly
#set1_dir = './sources/set1/'
#set2_dir = './sources/set2/'
#set3_dir = './sources/set3/'
#set4_dir = './sources/set4/'
datadir = './sources/'
outdir = './out/'

# Production data from WVGES

See website <http://www.wvgs.wvnet.edu/www/datastat/devshales.htm>  for lots of resources.  This includes
an excel sheet with lots of metadata on each unconventional well. We use that here to filter out conventional wells and for lat/lon, as well as monthly production.

**Note that the original file is in read-only lock.  You must make an explicit copy within Excel to successfully read in these data.**

In [3]:
klst = ['API','YEAR','G_JAN','G_FEB','G_MAR','G_APR','G_MAY','G_JUN','G_JUL','G_AUG','G_OCT','G_NOV','G_DEC',
        'O_JAN','O_FEB','O_MAR','O_APR','O_MAY','O_JUN','O_JUL','O_AUG','O_OCT','O_NOV','O_DEC',
        'NGL_JAN','NGL_FEB','NGL_MAR','NGL_APR','NGL_MAY','NGL_JUN','NGL_JUL','NGL_AUG','NGL_OCT','NGL_NOV','NGL_DEC',]
prod = pd.read_excel(datadir+'Copy of WVGES Marcellus Wells.xlsx',sheet_name='WV Marcellus Prod',
                     usecols=klst, skiprows=1)

The dataset has some records back to 1979 for working wells that are fracked in the early 2000's.  Presumably, the back records are included to demonstrate the difference between conventional and non-conventional.   I am leaving off years before 2002 (the earliest spud date) to keep the dataset from being swamped with empty cells.

In [4]:
prod = prod[prod.YEAR>2010]
prod.head()

,API,YEAR,G_JAN,G_FEB,G_MAR,G_APR,G_MAY,G_JUN,G_JUL,G_AUG,...,NGL_FEB,NGL_MAR,NGL_APR,NGL_MAY,NGL_JUN,NGL_JUL,NGL_AUG,NGL_OCT,NGL_NOV,NGL_DEC
3,4700102885,2011,748.0,717.0,617.0,590.0,590.0,689.0,636.0,512.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4700102885,2012,589.0,636.0,719.0,602.0,677.0,433.0,583.0,259.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4700102885,2013,650.0,488.0,514.0,519.0,747.0,648.0,689.0,513.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4700102885,2014,575.0,530.0,427.0,532.0,570.0,469.0,523.0,472.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4700102885,2015,448.0,501.0,403.0,351.0,567.0,356.0,283.0,381.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
prod.groupby("YEAR").count()

,API,G_JAN,G_FEB,G_MAR,G_APR,G_MAY,G_JUN,G_JUL,G_AUG,G_OCT,...,NGL_FEB,NGL_MAR,NGL_APR,NGL_MAY,NGL_JUN,NGL_JUL,NGL_AUG,NGL_OCT,NGL_NOV,NGL_DEC
YEAR,,,,,,,,,,,,,,,,,,,,,
2011,1802,1802,1802,1802,1802,1802,1802,1802,1802,1802,...,0,0,0,0,0,0,0,0,0,0
2012,2087,2087,2087,2087,2087,2087,2087,2087,2087,2087,...,0,0,0,0,0,0,0,0,0,0
2013,2476,2476,2476,2476,2476,2476,2476,2476,2476,2476,...,2476,2476,2476,2476,2476,2476,2476,2476,2476,2476
2014,2890,2890,2890,2890,2890,2890,2890,2890,2890,2890,...,2890,2890,2890,2890,2890,2890,2890,2890,2890,2890
2015,3062,3062,3062,3062,3062,3062,3062,3062,3062,3062,...,3062,3062,3062,3062,3062,3062,3062,3062,3062,3062
2016,3296,3296,3296,3296,3296,3296,3296,3296,3296,3296,...,1016,1016,1016,1016,1016,1016,1016,1016,1016,1016
2017,3589,3589,3589,3589,3589,3589,3589,3589,3589,3589,...,242,240,238,264,254,272,345,368,375,379
2018,1375,1375,1375,1375,1375,1375,1375,1375,1375,1375,...,1375,1375,1375,1375,1375,1375,1375,1375,1375,1375


In [4]:
xlate = {'JAN':'01', 'FEB':'02', 'MAR':'03', 'APR':'04', 'MAY':'05', 'JUN':'06',
         'JUL':'07', 'AUG':'08', 'SEP':'09', 'OCT':'10', 'NOV':'11', 'DEC':'12'}

def getColName(row):
    yr = str(row.YEAR)
    mo = xlate[row.variable[-3:]]
    return f'Y{yr}M{mo}'
    

mprod = prod.melt(id_vars=['API','YEAR'])
mprod = mprod.groupby(['API','YEAR','variable'],as_index=False)['value'].max()

mprod['typ'] = np.where(mprod.variable.str[0]=='G','GAS',np.nan)
mprod.typ = np.where(mprod.variable.str[0]=='O','OIL',mprod.typ)
mprod.typ = np.where(mprod.variable.str[0]=='N','NGL',mprod.typ)
mprod['yrmo'] = mprod.apply(lambda x: getColName(x),axis=1)
mprod.tail()

,API,YEAR,variable,value,typ,yrmo
855487,4710903065,2017,O_JUN,NaN,OIL,Y2017M06
855488,4710903065,2017,O_MAR,NaN,OIL,Y2017M03
855489,4710903065,2017,O_MAY,NaN,OIL,Y2017M05
855490,4710903065,2017,O_NOV,NaN,OIL,Y2017M11
855491,4710903065,2017,O_OCT,NaN,OIL,Y2017M10


In [6]:
mprod.to_pickle(outdir+'ver2.pkl')

# Metadata from WVGES

In [10]:
klst = ['API Number','County Name','Operator at Completion',
        'Surface Loc Longitude','Surface Loc Latitude','Spud Date']
meta = pd.read_excel(datadir+'Copy of WVGES Marcellus Wells.xlsx',sheet_name='Completed Marcellus Wells',
                     usecols=klst, skiprows=1)
meta.columns = ['API','CountyName','OperatorName',
        'Surf_Long','Surf_Lat','SpudDate']
meta['spud'] = pd.to_datetime(meta.SpudDate)
trimmed = meta[meta.spud.dt.year>2010]



In [14]:
len(trimmed)
trimmed.head()
trimmed.to_csv('WV_trimmed.csv')

In [7]:
print(f'Len meta: {len(meta)}, unique API: {len(meta.API.unique())}')

Len meta: 3718, unique API: 3703


In [8]:
#  Need to take most recent meta record:
meta = meta.groupby('API',as_index=False).last()

In [9]:
print(f'Len meta: {len(meta)}, unique API: {len(meta.API.unique())}')

Len meta: 3703, unique API: 3703
